# Stedin Distribution Transformer - Loss Model - Gridap Implementation

In [ ]:
using Gridap
using Gridap.Algebra;
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData

using GridapGmsh

using LinearAlgebra, ForwardDiff
using FillArrays, SparseArrays

# Geometry

In [ ]:
# Enclosure dimensions (enclosure is not centered, the core is)
hencl1 = 65.5e-2;   # Height above the x-axis
hencl2 = -53.5e-2;  # Height below the y-axis
wencl  = 104e-2;    # Width

# Core dimensions
wcore = 84e-2;
hcore = 100e-2;

# Core gap dimensions (left and right are identical)
wgap = 17e-2;
hgap = 76e-2;
mgap = 17e-2;

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Mesh densities
lc1 = 2e-2;      # Enclosure & core outer
lc2 = 1e-2;      # Core inner
lc3 = 1e-2;      # HV windings
lc4 = 1e-2;      # LV windings

# Gridap Setup

In [3]:
# Define norm of a complex vector for Gridap to use on the CellField which results from Bh = ∇(uh)
norm_complex(a::VectorValue{2, ComplexF64}) = sqrt(norm(a[1])^2 + norm(a[2])^2)
norm_complex(a::CellField) = Operation(norm_complex)(a)

norm_complex (generic function with 2 methods)

# Open-Circuit Test

In [4]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin_hybrid.msh")
Th = Triangulation(model)
Qh = CellQuadrature(Th, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
Vh = TestFESpace(model, reffe; conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
Uh = TrialFESpace(Vh, [0])

Info    : Reading 'geo/transformer_stedin_hybrid.msh'...
Info    : 194 entities
Info    : 11096 nodes
Info    : 17687 elements
Info    : Done reading 'geo/transformer_stedin_hybrid.msh'


TrialFESpace()

In [5]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [6]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Th)

const tag_core = get_tag_from_name(labels, "Core");
const tag_lv1l = get_tag_from_name(labels, "LV1l");
const tag_lv1r = get_tag_from_name(labels, "LV1r");
const tag_lv2l = get_tag_from_name(labels, "LV2l");
const tag_lv2r = get_tag_from_name(labels, "LV2r");
const tag_lv3l = get_tag_from_name(labels, "LV3l");
const tag_lv3r = get_tag_from_name(labels, "LV3r");

# Source current density
function fsource(tag)
    return 0.0 + 0.0im;
end

# Permeability model
bh_a = 1 / 47e3;
bh_b = 3.6;
bh_c = 2.1e8;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));

function fmur(tag, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fmur_core(normB);
    else
        return 1.0;
    end
end

function fnu(tag, ∇u)
    return 1 / (mu0 * fmur(tag, ∇u));
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.0;
    else
        return 0.0;
    end
end

fsigma (generic function with 1 method)

In [7]:
dv = get_fe_basis(Vh);
du = get_trial_fe_basis(Uh);

# Assemble the linear system
assem = SparseMatrixAssembler(Uh, Vh)
σk = get_cell_dof_ids(Uh);

In [8]:
# Circuit parameters
CFF  = 0.3;
Vsec = 420 * sqrt(2 / 3);
Rsec = 1.2999e-3 / CFF;
Asec = Awlv;
lsec = 0.4;

# Extend the linear system with an additional row and column per circuit equation
N  = num_free_dofs(Uh);
nc = 3;

C  = Matrix(UniformScaling(1), nc, nc);
f2 = [ Vsec * exp(1im * 2pi/3) / Rsec;
       Vsec / Rsec;
       Vsec * exp(-1im * 2pi/3) / Rsec
];

# Calculate the cell contributions to the extra vectors
Th_lv1r = Triangulation(model, tags = "LV1r"); Qh_lv1r = CellQuadrature(Th_lv1r, 2*order)
Th_lv1l = Triangulation(model, tags = "LV1l"); Qh_lv1l = CellQuadrature(Th_lv1l, 2*order)
Th_lv2r = Triangulation(model, tags = "LV2r"); Qh_lv2r = CellQuadrature(Th_lv2r, 2*order)
Th_lv2l = Triangulation(model, tags = "LV2l"); Qh_lv2l = CellQuadrature(Th_lv2l, 2*order)
Th_lv3r = Triangulation(model, tags = "LV3r"); Qh_lv3r = CellQuadrature(Th_lv3r, 2*order)
Th_lv3l = Triangulation(model, tags = "LV3l"); Qh_lv3l = CellQuadrature(Th_lv3l, 2*order)

cellvals_lv1lT = ∫( Ns / Asec ⋅ dv )*Qh_lv1l;
cellvals_lv1rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv1r;
cellvals_lv2lT = ∫( Ns / Asec ⋅ dv )*Qh_lv2l;
cellvals_lv2rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv2r;
cellvals_lv3lT = ∫( Ns / Asec ⋅ dv )*Qh_lv3l;
cellvals_lv3rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv3r;

cellvals_lv1l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1l;
cellvals_lv1r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1r;
cellvals_lv2l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2l;
cellvals_lv2r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2r;
cellvals_lv3l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3l;
cellvals_lv3r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3r;

# Assemble the circuit coupling equations into extra vectors
σ_lv1l = get_cell_dof_ids(Uh, Th_lv1l);
σ_lv1r = get_cell_dof_ids(Uh, Th_lv1r);
σ_lv2l = get_cell_dof_ids(Uh, Th_lv2l);
σ_lv2r = get_cell_dof_ids(Uh, Th_lv2r);
σ_lv3l = get_cell_dof_ids(Uh, Th_lv3l);
σ_lv3r = get_cell_dof_ids(Uh, Th_lv3r);

rs = ([cellvals_lv1lT; cellvals_lv1rT], [σ_lv1l; σ_lv1r]);
b1T = allocate_vector(assem, rs);
assemble_vector!(b1T, assem, rs);

rs = ([cellvals_lv2lT; cellvals_lv2rT], [σ_lv2l; σ_lv2r]);
b2T = allocate_vector(assem, rs);
assemble_vector!(b2T, assem, rs);

rs = ([cellvals_lv3lT; cellvals_lv3rT], [σ_lv3l; σ_lv3r]);
b3T = allocate_vector(assem, rs);
assemble_vector!(b3T, assem, rs);

rs = ([cellvals_lv1l; cellvals_lv1r], [σ_lv1l; σ_lv1r]);
b1 = allocate_vector(assem, rs);
assemble_vector!(b1, assem, rs);

rs = ([cellvals_lv2l; cellvals_lv2r], [σ_lv2l; σ_lv2r]);
b2 = allocate_vector(assem, rs);
assemble_vector!(b2, assem, rs);

rs = ([cellvals_lv3l; cellvals_lv3r], [σ_lv3l; σ_lv3r]);
b3 = allocate_vector(assem, rs);
assemble_vector!(b3, assem, rs);

In [9]:
u0 = FEFunction(Uh, zeros(Complex{Float64}, num_free_dofs(Uh)))

SingleFieldFEFunction():
 num_cells: 17463
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 9325442566621070826

In [10]:
# Calculate contributions of the cells to the linear system
cellvals_A = ∫( (fnu ∘ (τ, ∇(u0))) ⋅ ∇(du) ⋅ ∇(dv) + 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;

rs = ([cellvals_f], [σk]);
b = allocate_vector(assem, rs);
assemble_vector!(b, assem, rs);

rs = ([cellvals_A], [σk], [σk]);
A = allocate_matrix(assem, rs);
assemble_matrix!(A, assem, rs);

# Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
At = [A [b1T b2T b3T]; transpose([b1 b2 b3]) C];
bt = [b; f2];

# Solve the modified linear system and package the solution in a FEFunction
u = At \ bt;

In [ ]:
deltau = 1e6;
Niter = 1;
alpha = 0.9

tol = 1e-6;
Nmax = 200;

uhist = zeros(Complex{Float64}, N + nc);

while (deltau > tol) && (Niter < Nmax)
    uprev = u;
    uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

    uh_n = FEFunction(Uh, uhist)
    
    # Calculate contributions of the cells to the linear system
    cellvals_A = ∫( (fnu ∘ (τ, ∇(uh_n))) ⋅ ∇(du) ⋅ ∇(dv) + 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
    cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;
    
    rs = ([cellvals_f], [σk]);
    assemble_vector!(b, assem, rs);

    rs = ([cellvals_A], [σk], [σk]);
    assemble_matrix!(A, assem, rs);

    # Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
    At = [A [b1T b2T b3T]; transpose([b1 b2 b3]) C];
    bt = [b; f2];

    # Solve the modified linear system and package the solution in a FEFunction
    u = At \ bt;

    deltau = norm(u - uprev);
    Niter += 1;
    print("#$Niter: $deltau\n")
end

In [12]:
uh = FEFunction(Uh, u);

In [13]:
Isec1 = u[end-2];
Isec2 = u[end-1];
Isec3 = u[end-0];

print("I_{sec1}: ", norm(Isec1), " A, ", angle(Isec1) / pi * 180, " deg\n")
print("I_{sec2}: ", norm(Isec2), " A, ", angle(Isec2) / pi * 180, " deg\n")
print("I_{sec3}: ", norm(Isec3), " A, ", angle(Isec3) / pi * 180, " deg\n")

I_{sec1}: 28.270175411158533 A, -163.75398432823434 deg
I_{sec2}: 15.873064246026818 A, 89.8675404422131 deg
I_{sec3}: 28.358501221485763 A, -16.262128447019908 deg


In [14]:
# Post-processing for magnetic field and current density
Bh = norm_complex(∇(uh));
mu_r = fmur ∘ (τ, ∇(uh)); # Relative permeability

function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Ns * Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return Ns * -Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Ns * Isec2 / Awlv;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return Ns * -Isec2 / Awlv;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Ns * Isec3 / Awlv * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return Ns * -Isec3 / Awlv * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

# Post-processing for core losses
kv = 2.25; a = 1.53; b = 1.6;
#kv = 50; a = 1; b = 1.5;
kl = 0.955; # Lamination fill factor
K  = kv * 50^a;
L  = 0.4;

Thc = Triangulation(model, tags = "Core")
dΩc = Measure(Thc, 2*order);
Pv_core = K * Bh.^b;
Pcore = ∑(∫( kl * L * Pv_core )*dΩc) # Total core loss; take into account lamination fill factor

1109.0272025128838

In [15]:
writevtk(Th, "images/transformer_losses/transformer1", cellfields=["Az"=>abs(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy), "mur" => mu_r, "Pv" => Pv_core])

(["images/transformer_losses/transformer1.vtu"],)

![Flux Density](images/transformer_losses/oc_normB.png)

![Current Density](images/transformer_losses/oc_normJ.png)

# Short-Circuit Test

In [16]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin_hybrid.msh")
Th = Triangulation(model)
Qh = CellQuadrature(Th, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
Vh = TestFESpace(model, reffe; conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
Uh = TrialFESpace(Vh, [0])

Info    : Reading 'geo/transformer_stedin_hybrid.msh'...
Info    : 194 entities
Info    : 11096 nodes
Info    : 17687 elements
Info    : Done reading 'geo/transformer_stedin_hybrid.msh'


TrialFESpace()

In [17]:
Np = 266;
Ns = 6;

omega = 2*pi*50  # Frequency
mu0 = 4e-7 * pi;

In [18]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Th)

const tag_core = get_tag_from_name(labels, "Core");

const tag_hv1l = get_tag_from_name(labels, "HV1l");
const tag_hv1r = get_tag_from_name(labels, "HV1r");
const tag_hv2l = get_tag_from_name(labels, "HV2l");
const tag_hv2r = get_tag_from_name(labels, "HV2r");
const tag_hv3l = get_tag_from_name(labels, "HV3l");
const tag_hv3r = get_tag_from_name(labels, "HV3r");

const tag_lv1l = get_tag_from_name(labels, "LV1l");
const tag_lv1r = get_tag_from_name(labels, "LV1r");
const tag_lv2l = get_tag_from_name(labels, "LV2l");
const tag_lv2r = get_tag_from_name(labels, "LV2r");
const tag_lv3l = get_tag_from_name(labels, "LV3l");
const tag_lv3r = get_tag_from_name(labels, "LV3r");

# Source current density
function fsource(tag)
    return 0.0 + 0.0im;
end

# Permeability model
bh_a = 1 / 47e3;
bh_b = 3.6;
bh_c = 2.1e8;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));

function fmur(tag, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fmur_core(normB);
    else
        return 1.0;
    end
end

function fnu(tag, ∇u)
    return 1 / (mu0 * fmur(tag, ∇u));
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.0;
    else
        return 0.0;
    end
end

fsigma (generic function with 1 method)

In [19]:
dv = get_fe_basis(Vh);
du = get_trial_fe_basis(Uh);

# Assemble the linear system
assem = SparseMatrixAssembler(Uh, Vh)
σk = get_cell_dof_ids(Uh);

In [20]:
# Circuit parameters
CFFp = 0.3;
Vpri = 460.3 * sqrt(2);
Rpri = 1.8131 / CFFp;
Apri = Awhv;
lpri = 0.4;

CFFs = 0.3;
Vsec = 0;
Rsec = 1.2999e-3 / CFFs;
Asec = Awlv;
lsec = 0.4;

# Extend the linear system with an additional row and column per circuit equation
N  = num_free_dofs(Uh);
nc = 6;

C  = Matrix(UniformScaling(1), nc, nc);
f2 = [ 0; 0; 0;
       Vpri * exp(1im * 2pi/3) / Rpri;
       Vpri / Rpri;
       Vpri * exp(-1im * 2pi/3) / Rpri
];

# Calculate the cell contributions to the extra vectors
Th_lv1r = Triangulation(model, tags = "LV1r"); Qh_lv1r = CellQuadrature(Th_lv1r, 2*order)
Th_lv1l = Triangulation(model, tags = "LV1l"); Qh_lv1l = CellQuadrature(Th_lv1l, 2*order)
Th_lv2r = Triangulation(model, tags = "LV2r"); Qh_lv2r = CellQuadrature(Th_lv2r, 2*order)
Th_lv2l = Triangulation(model, tags = "LV2l"); Qh_lv2l = CellQuadrature(Th_lv2l, 2*order)
Th_lv3r = Triangulation(model, tags = "LV3r"); Qh_lv3r = CellQuadrature(Th_lv3r, 2*order)
Th_lv3l = Triangulation(model, tags = "LV3l"); Qh_lv3l = CellQuadrature(Th_lv3l, 2*order)

Th_hv1r = Triangulation(model, tags = "HV1r"); Qh_hv1r = CellQuadrature(Th_hv1r, 2*order)
Th_hv1l = Triangulation(model, tags = "HV1l"); Qh_hv1l = CellQuadrature(Th_hv1l, 2*order)
Th_hv2r = Triangulation(model, tags = "HV2r"); Qh_hv2r = CellQuadrature(Th_hv2r, 2*order)
Th_hv2l = Triangulation(model, tags = "HV2l"); Qh_hv2l = CellQuadrature(Th_hv2l, 2*order)
Th_hv3r = Triangulation(model, tags = "HV3r"); Qh_hv3r = CellQuadrature(Th_hv3r, 2*order)
Th_hv3l = Triangulation(model, tags = "HV3l"); Qh_hv3l = CellQuadrature(Th_hv3l, 2*order)

cellvals_lv1lT = ∫( Ns / Asec ⋅ dv )*Qh_lv1l;
cellvals_lv1rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv1r;
cellvals_lv2lT = ∫( Ns / Asec ⋅ dv )*Qh_lv2l;
cellvals_lv2rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv2r;
cellvals_lv3lT = ∫( Ns / Asec ⋅ dv )*Qh_lv3l;
cellvals_lv3rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv3r;

cellvals_hv1lT = ∫( -Np / Apri ⋅ dv )*Qh_hv1l;
cellvals_hv1rT = ∫( Np / Apri ⋅ dv )*Qh_hv1r;
cellvals_hv2lT = ∫( -Np / Apri ⋅ dv )*Qh_hv2l;
cellvals_hv2rT = ∫( Np / Apri ⋅ dv )*Qh_hv2r;
cellvals_hv3lT = ∫( -Np / Apri ⋅ dv )*Qh_hv3l;
cellvals_hv3rT = ∫( Np / Apri ⋅ dv )*Qh_hv3r;

cellvals_lv1l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1l;
cellvals_lv1r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1r;
cellvals_lv2l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2l;
cellvals_lv2r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2r;
cellvals_lv3l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3l;
cellvals_lv3r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3r;

cellvals_hv1l = ∫( 1im * omega * Np * lpri / (Apri * Rpri) ⋅ dv )*Qh_hv1l;
cellvals_hv1r = ∫( -1im * omega * Np * lpri / (Apri * Rpri) ⋅ dv )*Qh_hv1r;
cellvals_hv2l = ∫( 1im * omega * Np * lpri / (Apri * Rpri) ⋅ dv )*Qh_hv2l;
cellvals_hv2r = ∫( -1im * omega * Np * lpri / (Apri * Rpri) ⋅ dv )*Qh_hv2r;
cellvals_hv3l = ∫( 1im * omega * Np * lpri / (Apri * Rpri) ⋅ dv )*Qh_hv3l;
cellvals_hv3r = ∫( -1im * omega * Np * lpri / (Apri * Rpri) ⋅ dv )*Qh_hv3r;

# Assemble the circuit coupling equations into extra vectors
σ_lv1l = get_cell_dof_ids(Uh, Th_lv1l);
σ_lv1r = get_cell_dof_ids(Uh, Th_lv1r);
σ_lv2l = get_cell_dof_ids(Uh, Th_lv2l);
σ_lv2r = get_cell_dof_ids(Uh, Th_lv2r);
σ_lv3l = get_cell_dof_ids(Uh, Th_lv3l);
σ_lv3r = get_cell_dof_ids(Uh, Th_lv3r);

σ_hv1l = get_cell_dof_ids(Uh, Th_hv1l);
σ_hv1r = get_cell_dof_ids(Uh, Th_hv1r);
σ_hv2l = get_cell_dof_ids(Uh, Th_hv2l);
σ_hv2r = get_cell_dof_ids(Uh, Th_hv2r);
σ_hv3l = get_cell_dof_ids(Uh, Th_hv3l);
σ_hv3r = get_cell_dof_ids(Uh, Th_hv3r);

rs = ([cellvals_lv1lT; cellvals_lv1rT], [σ_lv1l; σ_lv1r]);
b1T = allocate_vector(assem, rs);
assemble_vector!(b1T, assem, rs);

rs = ([cellvals_lv2lT; cellvals_lv2rT], [σ_lv2l; σ_lv2r]);
b2T = allocate_vector(assem, rs);
assemble_vector!(b2T, assem, rs);

rs = ([cellvals_lv3lT; cellvals_lv3rT], [σ_lv3l; σ_lv3r]);
b3T = allocate_vector(assem, rs);
assemble_vector!(b3T, assem, rs);

rs = ([cellvals_hv1lT; cellvals_hv1rT], [σ_hv1l; σ_hv1r]);
b4T = allocate_vector(assem, rs);
assemble_vector!(b4T, assem, rs);

rs = ([cellvals_hv2lT; cellvals_hv2rT], [σ_hv2l; σ_hv2r]);
b5T = allocate_vector(assem, rs);
assemble_vector!(b5T, assem, rs);

rs = ([cellvals_hv3lT; cellvals_hv3rT], [σ_hv3l; σ_hv3r]);
b6T = allocate_vector(assem, rs);
assemble_vector!(b6T, assem, rs);

rs = ([cellvals_lv1l; cellvals_lv1r], [σ_lv1l; σ_lv1r]);
b1 = allocate_vector(assem, rs);
assemble_vector!(b1, assem, rs);

rs = ([cellvals_lv2l; cellvals_lv2r], [σ_lv2l; σ_lv2r]);
b2 = allocate_vector(assem, rs);
assemble_vector!(b2, assem, rs);

rs = ([cellvals_lv3l; cellvals_lv3r], [σ_lv3l; σ_lv3r]);
b3 = allocate_vector(assem, rs);
assemble_vector!(b3, assem, rs);

rs = ([cellvals_hv1l; cellvals_hv1r], [σ_hv1l; σ_hv1r]);
b4 = allocate_vector(assem, rs);
assemble_vector!(b4, assem, rs);

rs = ([cellvals_hv2l; cellvals_hv2r], [σ_hv2l; σ_hv2r]);
b5 = allocate_vector(assem, rs);
assemble_vector!(b5, assem, rs);

rs = ([cellvals_hv3l; cellvals_hv3r], [σ_hv3l; σ_hv3r]);
b6 = allocate_vector(assem, rs);
assemble_vector!(b6, assem, rs);

In [21]:
u0 = FEFunction(Uh, zeros(Complex{Float64}, num_free_dofs(Uh)))

SingleFieldFEFunction():
 num_cells: 17463
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10094870957440362822

In [22]:
# Calculate contributions of the cells to the linear system
cellvals_A = ∫( (fnu ∘ (τ, ∇(u0))) ⋅ ∇(du) ⋅ ∇(dv) + 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;

rs = ([cellvals_f], [σk]);
b = allocate_vector(assem, rs);
assemble_vector!(b, assem, rs);

rs = ([cellvals_A], [σk], [σk]);
A = allocate_matrix(assem, rs);
assemble_matrix!(A, assem, rs);

# Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
At = [A [b1T b2T b3T b4T b5T b6T]; transpose([b1 b2 b3 b4 b5 b6]) C];
bt = [b; f2];

# Solve the modified linear system and package the solution in a FEFunction
u = At \ bt;

In [23]:
deltau = 1e6;
Niter = 1;
alpha = 0.9

tol = 1e-6;
Nmax = 200;

uhist = zeros(Complex{Float64}, N + nc);

while (deltau > tol) && (Niter < Nmax)
    uprev = u;
    uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

    uh_n = FEFunction(Uh, u)
    
    # Calculate contributions of the cells to the linear system
    cellvals_A = ∫( (fnu ∘ (τ, ∇(uh_n))) ⋅ ∇(du) ⋅ ∇(dv) + 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
    cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;
    
    rs = ([cellvals_f], [σk]);
    assemble_vector!(b, assem, rs);

    rs = ([cellvals_A], [σk], [σk]);
    assemble_matrix!(A, assem, rs);

    # Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
    At = [A [b1T b2T b3T b4T b5T b6T]; transpose([b1 b2 b3 b4 b5 b6]) C];
    bt = [b; f2];

    # Solve the modified linear system and package the solution in a FEFunction
    u = At \ bt;

    deltau = norm(u - uprev);
    Niter += 1;
    print("#$Niter: $deltau\n")
end

#2: 8.506184760767698e-10


In [24]:
uh = FEFunction(Uh, u);

In [25]:
Isec1 = u[end-5];
Isec2 = u[end-4];
Isec3 = u[end-3];

Ipri1 = u[end-2];
Ipri2 = u[end-1];
Ipri3 = u[end-0];

print("I_{pri1}: ", norm(Ipri1), " A, ", angle(Ipri1) / pi * 180, " deg\n")
print("I_{pri2}: ", norm(Ipri2), " A, ", angle(Ipri2) / pi * 180, " deg\n")
print("I_{pri3}: ", norm(Ipri3), " A, ", angle(Ipri3) / pi * 180, " deg\n")

print("I_{sec1}: ", norm(Isec1), " A, ", angle(Isec1) / pi * 180, " deg\n")
print("I_{sec2}: ", norm(Isec2), " A, ", angle(Isec2) / pi * 180, " deg\n")
print("I_{sec3}: ", norm(Isec3), " A, ", angle(Isec3) / pi * 180, " deg\n")

I_{pri1}: 44.603179165658986 A, 123.08065854785949 deg
I_{pri2}: 44.65219604627557 A, 3.0552798664346574 deg
I_{pri3}: 44.678905268710906 A, -116.89898226096406 deg
I_{sec1}: 1980.601322519662 A, 123.04682521523588 deg
I_{sec2}: 1978.726303469108 A, 3.143913991592181 deg
I_{sec3}: 1978.4069045090428 A, -116.97597841227613 deg


In [26]:
Rpri = 1.8131;
Rsec = 1.2999e-3;

Ppri = norm(Ipri1)^2 * Rpri/2 + norm(Ipri2)^2 * Rpri/2 + norm(Ipri3)^2 * Rpri/2;
Psec = norm(Isec1)^2 * Rsec/2 + norm(Isec2)^2 * Rsec/2 + norm(Isec3)^2 * Rsec/2;

print("Primary loss: $Ppri W\n")
print("Secondary loss: $Psec W\n")

Primary loss: 5420.6856089098055 W
Secondary loss: 7638.364009037405 W


In [27]:
# Post-processing for magnetic field and current density
Bh = norm_complex(∇(uh));
mu_r = fmur ∘ (τ, ∇(uh)); # Relative permeability

function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Ns * Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return Ns * -Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Ns * Isec2 / Awlv;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return Ns * -Isec2 / Awlv;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Ns * Isec3 / Awlv * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return Ns * -Isec3 / Awlv * exp(-1im * 2pi/3);
    elseif tag == tag_hv1l        # LV winding phase 1 left
        return Np * -Ipri1 / Awhv * exp(1im * 2pi/3);
    elseif tag == tag_hv1r    # LV winding phase 1 right
        return Np * Ipri1 / Awhv * exp(1im * 2pi/3);
    elseif tag == tag_hv2l    # LV winding phase 2 left
        return Np * -Ipri2 / Awhv;
    elseif tag == tag_hv2r    # LV winding phase 2 right
        return Np * Ipri2 / Awhv;
    elseif tag == tag_hv3l    # LV winding phase 3 left
        return Np * -Ipri3 / Awhv * exp(-1im * 2pi/3);
    elseif tag == tag_hv3r    # LV winding phase 3 right
        return Np * Ipri3 / Awhv * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [28]:
writevtk(Th, "images/transformer_losses/transformer2", cellfields=["Az"=>abs(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy), "mur" => mu_r])

(["images/transformer_losses/transformer2.vtu"],)

![Flux Density](images/transformer_losses/sc_normB.png)

![Current Density](images/transformer_losses/sc_normJ.png)